In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig

bin D:\my_projects\saiga_mistral_333\venv\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [2]:
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"

In [3]:
class Conversation:
    def __init__(self,
                 message_template=DEFAULT_MESSAGE_TEMPLATE,
                 system_prompt=DEFAULT_SYSTEM_PROMPT,
                 response_template=DEFAULT_RESPONSE_TEMPLATE):
        
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{"role": "system", "content": system_prompt}]

    def add_user_message(self, message):
        self.messages.append({"role": "user", "content": message})

    def add_bot_message(self, message):
        self.messages.append({"role": "bot", "content": message})

    def get_prompt(self) -> str:
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()

In [4]:
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [5]:
config = PeftConfig.from_pretrained(MODEL_NAME)

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                             quantization_config=bnb_config,
                                             torch_dtype=torch.float16,
                                             device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model = PeftModel.from_pretrained(model,
                                  MODEL_NAME,
                                  torch_dtype=torch.float16)
model.eval();

In [7]:
model.device

device(type='cuda', index=0)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



### generation

In [9]:
inputs = ["Почему трава зеленая?"]
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt()
    
    print(f'CURRENT PROMT: {prompt}')

    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

CURRENT PROMT: <s>system
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.</s><s>user
Почему трава зеленая?</s><s>bot
Почему трава зеленая?
Зеленый цвет у растений обусловлен наличием в них химических веществ, известных как хлорофиллы. Хлорофиллы являются пигментами, которые позволяют растениям собирать энергию из света для фотосинтеза - процесса, благодаря которому они могут производить свою пищу из воды и углекислого газа.

Хлорофиллы имеют особые структуры, которые позволяют им поглощать свет в определенном диапазоне длин волн, что создает видимый цвет растительности. Зеленый цвет является результатом поглощения световых лучей с длинами волн 430-560 нм, которые не поглощаются другими пигментами, такими как каротиноиды или антоцианы, которые придают красный и жёлтый цвет соответственно.

Таким образом, зеленый цвет растений является результатом специфического поглощения световых лучей хлорофиллами, которые позволяют растениям проводить фот

In [10]:
bank_prompt = ["""
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им в управлении банком.

Описание банка: 
внутренняя организация работы кредитного учреждения, с помощью которой структурируются и формализуются подходы и методы управления, 
определяются группы исполнителей, разрабатываются системы контроля и внутриорганизационных взаимоотношений.
С помощью организационной структуры осуществляются все необходимые действия, направленные на достижение целей, которые ставит перед собой банк.
В современном менеджменте существует несколько моделей организационных структур, которые используют кредитные организации.
Механистическая структура наиболее традиционная. 
Каждое отдельное подразделение специализируются на своем круге операций. Механистические структуры бывают двух видов:
1. Функциональные (или линейно-функциональные) подразделения создаются для решения определенных задач. При такой структуре в крупных кредитных организациях создается вертикальная иерархия департаментов, которые делятся на управления, а те в свою очередь – на отделы, отделы – на сектора, сектора – на группы и т. д. Функциональное деление чаще всего используется небольшими и средними кредитными организациями. В его основе – пооперационное разделение структур. Например, создаются валютный, кредитный, операционный отделы и т. п.
2. Дивизионная структура, при которой деление организации ориентировано на потребителя, продукт или регион. 
В обязанности сотрудников соответствующих управлений входит полное обслуживание клиентов по всем видам оказываемых банком услуг. 
Они обязаны уметь объяснить правила предоставления услуг и дать свои рекомендации по возможным операциям.
Чаще всего структурно сотрудники поделены на два-три управления: 
по работе с физическими лицами, обслуживанию юридических лиц, иногда создается специальное подразделение для VIP-клиентов. 
При дивизионной структуре существует разделение на фронт-офис, отвечающий за работу с клиентами, проведение операций, и бэк-офис, 
на который возложена работа по оформлению документации и другое обслуживание работы фронт-офиса.
Органические структуры – альтернатива механистическим – ориентированы на быстрые изменения окружающей среды. 
К этой группе относятся такие структуры:
– проектная, действует для реализации отдельно взятого проекта;
– матричная, представляет собой совмещение традиционных механистических структур с проектным подходом к реализации целей.
Организационная структура кредитных организаций не регулируется законодательно. Каждый  вправе выбирать собственную систему."""]

In [11]:
inputs = ['Сформируй организационную структуру банка, в формате JSON.']

for inp in inputs:
    conversation = Conversation(system_prompt=bank_prompt)
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt()
    
    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

    conversation.add_bot_message(output)

Сформируй организационную структуру банка, в формате JSON.
{
    "name": "Банк",
    "departments": [
        {
            "name": "Финансовый отдел",
            "responsibilities": ["управление финансовыми ресурсами", "планирование и контроль расходов"],
            "subdepartments": [
                {
                    "name": "Управление капиталом",
                    "responsibilities": ["инвестиции", "финансирование", "стратегический анализ"]
                },
                {
                    "name": "Управление бюджетом",
                    "responsibilities": ["планирование и контроль расходов", "анализ экономической ситуации", "финансовые планы"]
                }
            ]
        },
        {
            "name": "Отдел маркетинга",
            "responsibilities": ["разработка стратегии развития", "проведение маркетинговых кампаний", "анализ рынка"],
            "subdepartments": [
                { "name": "Аналитический отдел", "responsibilities": ["анализ д

In [12]:
inputs = ['Добавь в финансовый отдел "Управление рисками".']

for inp in inputs:
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt()
    
    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

    conversation.add_bot_message(output)

Добавь в финансовый отдел "Управление рисками".
{
    "name": "Банк"
    "departments": [
        {
            "department": "Финансовый отдел",
            "subdepartments": [
                {
                "name": "Управление капиталом",
                "responsibilities": ["инвестиции", "финансировние", "стратегический анализ"]
                }
                {
                "name": "Управление бюджетом",
                "responsibilities": ["планирование и контроль расход", "анализ экономической ситуации", "financial plans"]
                }
                {
                "name":"Управление рисков",
                "responsibilities":["анализ рисков", "управление рисками", "стратегия минимизации рисков"]
                }
            ]
        }
    ]
}


